<a href="https://colab.research.google.com/github/MauriVass/ML4IoTCourse/blob/master/Lab5/RESTfulInfKeywordSpottingService.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import argparse
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

class Preprocess:
  def __init__(self, sampling_rate, frame_length, frame_step, num_mel_bins=None, lower_frequency=None, upper_frequency=None, num_coefficients=None, mfcc=False, image_size=32):
    self.sampling_rate=sampling_rate
    self.frame_length=frame_length
    self.frame_step=frame_step
    self.num_mel_bins = num_mel_bins
    self.lower_frequency = lower_frequency
    self.upper_frequency = upper_frequency
    self.num_coefficients = num_coefficients
    self.mfccs=mfcc
    self.image_size = image_size

    if(mfcc):
      num_spectrogram_bins = frame_length // 2 + 1
      self.linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
          self.num_mel_bins,
          num_spectrogram_bins,
          self.sampling_rate,
          self.lower_frequency,
          self.upper_frequency)
      self.preprocess = self.preprocess_with_mfcc
    else:
      self.preprocess = self.preprocess_with_stft


  def read(self, file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    label = parts[-2]
    label_id = tf.argmax(label == self.labels)
    audio_bynary = tf.io.read_file(file_path)
    audio, _ = tf.audio.decode_wav(audio_bynary)
    #print('Sampling: ', np.array(r))
    audio = tf.squeeze(audio, axis=1)
    return audio, label_id

  def pad(self, audio):
    zero_padding = tf.zeros(self.sampling_rate - tf.shape(audio), dtype=tf.float32)
    audio = tf.concat([audio,zero_padding],0)
    audio.set_shape([self.sampling_rate])
    return audio

  def get_spectrogram(self, audio):
    #Calculate the STFT of the signal given frame_length and frame_step
    stft = tf.signal.stft(audio,
            frame_length=self.frame_length,
            frame_step=self.frame_step,
            fft_length=self.frame_length)
    #Transform the complex number in real number
    spectrogram = tf.abs(stft)
    return spectrogram

  def get_mfccs(self, spectrogram):
    mel_spectrogram = tf.tensordot(spectrogram,
            self.linear_to_mel_weight_matrix, 1)
    log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
    mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
    mfccs = mfccs[:, :self.num_coefficients]
    return mfccs

  def preprocess_with_stft(self, audio):
    #audio, label = self.read(file_path)
    audio = self.pad(audio)
    spectrogram = self.get_spectrogram(audio)
    spectrogram = tf.expand_dims(spectrogram, -1)
    spectrogram  = tf.image.resize(spectrogram, [self.image_size,self.image_size])
    return spectrogram, label

  def preprocess_with_mfcc(self, audio):
    #audio, label = self.read(file_path)
    audio = self.pad(audio)
    spectrogram = self.get_spectrogram(audio)
    mfccs = self.get_mfccs(spectrogram)
    mfccs = tf.expand_dims(mfccs, -1)
    return mfccs, label

  def preprocess(self, audio):
    #This method creates a dataset from a numpy array (our listfile path)
    #ds = tf.data.Dataset.from_tensor_slices(files)
    #Different preprocess step depending on the input parameter
    # ds = ds.map(self.preprocess, num_parallel_calls=4)
    # ds = ds.batch(32)
    # ds = ds.cache()

    # if(train is True):
    # 	ds = ds.shuffle(100, reshuffle_each_iteration=True)
    audio = self.preprocess(audio)
    return audio

In [2]:
import tensorflow.lite as tflite

def Evaluate(model,audio):
  saved_model_dir = model

  interpreter = tf.lite.Interpreter(model_path=saved_model_dir)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  
  interpreter.set_tensor(input_details[0]['index'], audio)
  interpreter.invoke()
  output = interpreter.get_tensor(output_details[0]['index'])[0]

  LABELS = ["go", "left", "no", "right", "stop", "up", "yes"]
  index_pred = np.argmax(output)
  
  prediction = LABELS[index_pred]
  confidence = output[index_pred]
  return prediction, confidence

In [3]:
!pip install cherrypy

     |████████████████████████████████| 419kB 5.6MB/s 
     |████████████████████████████████| 102kB 8.9MB/s 


In [ ]:
import cherrypy
import sys

class HelloWorld:
    def index(self):
        return "Hello World!"
    index.exposed = True
if __name__ == '__main__':
   config = {'server.socket_host': '0.0.0.0','server.socket_port' : 18888}
   cherrypy.config.update(config)
   cherrypy.quickstart(HelloWorld())

In [ ]:
import random
import string
import cherrypy
import json 

class KSInferenceService(object):
  #Required to be accessable online
  exposed=True

  # def POST(self,*path,**query):
  #   return json.dumps(output)

	# def POST(self,*path,**query):
	# 	return

  def POST(self,*path,**query):
    models = ['MLP', 'CNN', 'DSCNN']
    if(len(path)!=1):
      raise cherrypy.HTTPError(404,f"Use only 1 model: {models}. Used: {path}")
    model = str(path[0])
    #Command not expected
    if(model not in models):
      raise cherrypy.HTTPError(404,f"Model not recognized. Use: {models}. Used: {model}")

    input = cherrypy.request.body.read()
    input = json.loads(input)

    audio = signal.resample_poly(audio,1,3)
    mfcc = True
    sampling_rate = 16000
    frame_length = 640  #Default 640 (mfcc=True), 256(mfcc=False)
    frame_step = 320 #Default 320 (mfcc=True), 128(mfcc=False)
    num_mel_bins = 40 #Default 40 (only mfcc=True)
    num_coefficients = 10 #Default 10 (only mfcc=True)
    Preprocess(sampling_rate, frame_length, frame_step, num_mel_bins, lower_frequency, upper_frequency, num_coefficients, mfcc, image_size=32)

    pred, conf = Evaluate(model,audio)

    output = {'prediction':pred, 'confidence':conf}

    return json.dumps(output)

	# def DELETE(self,*path,**query):
	# 	return

#'request.dispatch': cherrypy.dispatch.MethodDispatcher() => switch from default URL to HTTP compliant approch
conf = { '/': {	'request.dispatch': cherrypy.dispatch.MethodDispatcher(),
								'tools.sessions.on':True} 
					}
cherrypy.tree.mount(KSInferenceService(), '/', conf)

cherrypy.config.update({'servet.socket_host':'0.0.0.0'})
cherrypy.config.update({'servet.socket_port':'8888'})

### ### ###
##Start a server in colab
#bind the port 8888 and get a weblink to access
# from google.colab.output import eval_js
# print(eval_js("google.colab.kernel.proxyPort(8888)"))

# #run the script/API in the background
# import subprocess
# subprocess.Popen(["python", "/", "8888"]) 
# cherrypy.quickstart(KSInferenceService())
### ### ###

#cherrypy.engine.start()
#cherrypy.engine.block()

[27/Dec/2020:14:37:50] ENGINE Listening for SIGTERM.
[27/Dec/2020:14:37:50] ENGINE Listening for SIGHUP.
[27/Dec/2020:14:37:50] ENGINE Listening for SIGUSR1.
[27/Dec/2020:14:37:50] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[27/Dec/2020:14:37:50] ENGINE Started monitor thread 'Autoreloader'.


https://pdwn63jxbsj-496ff2e9c6d22116-8888-colab.googleusercontent.com/


[27/Dec/2020:14:37:51] ENGINE Error in 'start' listener <bound method Server.start of <cherrypy._cpserver.Server object at 0x7f4215d539b0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/portend.py", line 122, in free
    Checker(timeout=0.1).assert_free(host, port)
  File "/usr/local/lib/python3.6/dist-packages/portend.py", line 71, in assert_free
    list(itertools.starmap(self._connect, info))
  File "/usr/local/lib/python3.6/dist-packages/portend.py", line 87, in _connect
    raise PortNotFree(tmpl.format(**locals()))
portend.PortNotFree: Port 127.0.0.1 is in use on 8080.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/cherrypy/process/wspbus.py", line 230, in publish
    output.append(listener(*args, **kwargs))
  File "/usr/local/lib/python3.6/dist-packages/cherrypy/_cpserver.py", line 180, in start
    super(Server, self).start()
  File "/usr/